<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/Bidirectional_RNNs_and_Beam_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In a previous notebook, we explained an important NLP task: [*neural machine translation*](https://github.com/victorviro/Deep_learning_python/blob/master/NLP_Encoder_Decoder_NMT.ipynb) (NMT), using a pure Encoder-Decoder model. Let's see how we can improve this architecture using bidirectional RNNs or the beam search algorithm.

## Bidirectional RNNs


At each time step, a regular recurrent layer only looks at past and present inputs before generating its output. In other words, it is “causal,” meaning it cannot look into the future. This type of RNN makes sense when forecasting time series, but for many NLP tasks, such as Neural Machine Translation, it is often preferable to look ahead at the next words before encoding a given word. For example, consider the phrases "the Queen of the United Kingdom", "the queen of hearts", and "the queen bee": to properly encode the word "queen", we need to look ahead. To implement this, we can run two recurrent layers on the same inputs, one reading the words from left to right and the other reading them from right to left. Then we simply combine their outputs at each time step, typically by concatenating them. This is called a *bidirectional recurrent layer* (see Figure 16-5).

![texto alternativo](https://i.ibb.co/p1cFm9V/bidirectional-recurrent-layer.png)

For a further explanation of how bidirectional RNNs work you can check this [video](https://www.coursera.org/lecture/nlp-sequence-models/bidirectional-rnn-fyXnn).

To implement a bidirectional recurrent layer in Keras, we can wrap a recurrent layer in a `keras.layers.Bidirectional` layer. For example, the following code creates a bidirectional GRU layer:

In [ ]:
import keras
keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))

**Note**: The `Bidirectional` layer will create a clone of the `GRU` layer (but in the reverse direction), and it will run both and concatenate their outputs. So although the `GRU` layer has 10 units, the `Bidirectional` layer will output 20 values per time step.


In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, None, 10)          660       
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 20)          1320      
Total params: 1,980
Trainable params: 1,980
Non-trainable params: 0
_________________________________________________________________


## Beam Search

Suppose we train an Encoder-Decoder model and use it to translate the French sentence "Comment vas-tu?" to English. We are hoping that it will output the proper translation ("How are you?"), but unfortunately, it outputs "How will you?" Looking at the training set, we notice many sentences such as "Comment vas-tu jouer?" which translates to "How will you play?" So it wasn’t absurd for the model to output "How will" after seeing "Comment vas". Unfortunately, in this case, it was a mistake, and the model could not go back and fix it, so it tried to complete the sentence as best it could. By greedily outputting the most likely word at every step, it ended up with a suboptimal translation. How can we give the model a chance to go back and fix mistakes it made earlier? One of the most common solutions is *beam search*: it keeps track of a short list of the `k` most promising sentences (say, the top three), and at each decoder step it tries to extend them by one word, keeping only the k most likely sentences. The parameter `k` is called the *beam width*.

For example, suppose we use the model to translate the sentence "Comment vas-tu?" using beam search with a beam width of 3. At the first decoder step, the model will output an estimated probability for each possible word. Suppose the top three words are "How" (75% estimated probability), "What" (3%), and "You" (1%). That’s our short list so far. Next, we create three copies of our model and use them to find the next word for each sentence. Each model will output one estimated probability per word in the vocabulary. The first model will try to find the next word in the sentence "How", and perhaps it will output a probability of 36% for the word "will", 32% for the word "are", 16% for the word "do", and so on. Note that these are actually *conditional* probabilities, given that the sentence starts with "How". The second model will try to complete the sentence "What"; it might output a conditional probability of 50% for the word "are", and so on. Assuming the vocabulary has 10000 words, each model will output 10000 probabilities.

Next, we compute the probabilities of each of the 30000 two-word sentences that these models considered ($3 \times 10000$). We do this by multiplying the estimated conditional probability of each word by the estimated probability of the sentence it completes. For example, the estimated probability of the sentence "How" was 75%, while the estimated conditional probability of the word "will" (given that the first word is "How") was 36%, so the estimated probability of the sentence "How will" is $75 \times 36 = 27$%. After computing the probabilities of all 30000 two-word sentences, we keep only the top 3. Perhaps they all start with the word "How": "How will" (27%), "How are" (24%), and "How do" (12%). Right now, the sentence "How will" is winning, but "How are" has not been eliminated.

Then we repeat the same process: we use three models to predict the next word in each of these three sentences, and we compute the probabilities of all 30000 three-word sentences we considered. Perhaps the top three are now "How are you" (10%), "How do you" (8%), and "How will you" (2%). At the next step we may get "How do you do" (7%), "How are you `<eos>`" (6%), and "How are you doing" (3%). Notice that "How will" was eliminated, and we now have three perfectly reasonable translations. We boosted our Encoder-Decoder model’s performance without any extra training, simply by using it more wisely.

![](https://i.ibb.co/VmCfXvY/beam-search.png)


For a further explanation of how beam search works you can check this [video](https://www.coursera.org/lecture/nlp-sequence-models/beam-search-4EtHZ).

We can implement beam search on an encoder-decoder network for Neural Machine Translation fairly easily using TensorFlow Addons (see [post](https://github.com/tensorflow/addons/pull/2140)). We can get good translations for fairly short sentences (especially if we use pre-trained word embeddings). 

Unfortunately, this model will be really bad at translating long sentences. Once again, the problem comes from the limited short-term memory of RNNs. *Attention mechanisms* are the game-changing innovation that addressed this problem. We will see it in the next notebook.

# References

- [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

- https://github.com/ageron/handson-ml2

- [Bidirectional RNNs Coursera](https://www.coursera.org/lecture/nlp-sequence-models/bidirectional-rnn-fyXnn)

- [Beam Search Coursera](https://www.coursera.org/lecture/nlp-sequence-models/beam-search-4EtHZ)

- [Beam search with TF Addons](https://github.com/tensorflow/addons/pull/2140)

- [Beam Search post](https://d2l.ai/chapter_recurrent-modern/beam-search.html)

